# Köpek Barınağı Yönetim Sistemi

### Gerekli kütüphaneleri içe aktar

In [1]:
import pandas as pd
from pandasql import sqldf
from flask import Flask, render_template, request, redirect, url_for
import webbrowser
import random
from openpyxl import load_workbook
import sqlite3

In [2]:
app = Flask(__name__)

### Locals yazmayı bıraktıran fonksiyon

In [3]:
def pysqldf(q):
  return sqldf(q, globals())

### Excelleri oku

In [4]:
filePath = './Data/'  

In [5]:
dfAnimals = pd.read_excel(filePath + 'data.xlsx',sheet_name="hayvan")
dfShelters = pd.read_excel(filePath + 'data.xlsx',sheet_name="barinak")
dfEmployees = pd.read_excel(filePath + 'data.xlsx',sheet_name="personel")
dfVets = pd.read_excel(filePath + 'data.xlsx',sheet_name="veteriner")
dfInspections = pd.read_excel(filePath + 'data.xlsx',sheet_name="muayene")

### Excelleri veritabanına çevir

In [6]:
conn = sqlite3.connect('veritabani.db')

dfAnimals.to_sql('hayvan', conn, if_exists='replace', index=False)
dfShelters.to_sql('barinak', conn, if_exists='replace', index=False)
dfInspections.to_sql('muayene', conn, if_exists='replace', index=False)
dfEmployees.to_sql('personel', conn, if_exists='replace', index=False)
dfVets.to_sql('veteriner', conn, if_exists='replace', index=False)


# ÖRNEK
result = pd.read_sql_query("SELECT * FROM hayvan", conn)
conn.close()

result

,HayvanId,Ad,Tur,Cinsiyet,Yas,SaglikDurumu,GelisTarihi,BarinakId
0,1,Gizmo,Clumber Spaniel,Erkek,8,Saglikli,2023-11-22 00:00:00,1
1,2,Kiki,Australian Cattle Dog,Disi,13,Tedavi Ediliyor,2021-01-25 00:00:00,1
2,3,Olaf,American Foxhound,Erkek,12,Tedavi Ediliyor,2022-10-08 00:00:00,1
3,4,Cleo,Irish Setter,Erkek,3,Tedavi Ediliyor,2023-07-25 00:00:00,1
4,5,Lucy,Afghan Hound,Disi,11,Tedavi Ediliyor,2023-01-22 00:00:00,1
...,...,...,...,...,...,...,...,...
496,497,Olaf,Welsh Terrier,Erkek,8,Tedavi Ediliyor,2024-07-22 00:00:00,27
497,498,Bella,Chihuahua,Disi,1,Hasta,2020-05-26 00:00:00,27
498,499,Vader,American Eskimo Dog,Disi,7,Hasta,2020-06-02 00:00:00,27
499,500,Buster,Belgian Sheepdog,Disi,7,Tedavi Ediliyor,2023-02-12 00:00:00,28


### Excellerden ilk 5 satır

In [7]:
dfAnimals.head(5)

,HayvanId,Ad,Tur,Cinsiyet,Yas,SaglikDurumu,GelisTarihi,BarinakId
0,1,Gizmo,Clumber Spaniel,Erkek,8,Saglikli,2023-11-22,1
1,2,Kiki,Australian Cattle Dog,Disi,13,Tedavi Ediliyor,2021-01-25,1
2,3,Olaf,American Foxhound,Erkek,12,Tedavi Ediliyor,2022-10-08,1
3,4,Cleo,Irish Setter,Erkek,3,Tedavi Ediliyor,2023-07-25,1
4,5,Lucy,Afghan Hound,Disi,11,Tedavi Ediliyor,2023-01-22,1


In [8]:
dfShelters.head()

,BarinakId,Ad,Konum,Kapasite,Sifre
0,1,Cankaya Karatas Gecici Hayvan Bakimevi,"Cankaya, Ankara",6000,1
1,2,Golbasi Gecici Hayvan Bakimevi ve Rehabilitasy...,"Gölbasi, Ankara",1500,2
2,3,Sincan Gecici Hayvan Bakimevi ve Rehabilitasyo...,"Sincan, Ankara",2200,3
3,4,Patipark Hayvan Barinagi,"Ayas, Ankara",1000,4
4,5,Etimesgut Hayvan Barinagi,"Etimesgut, Ankara",200,5


In [9]:
dfEmployees.head()

,PersonelId,Ad,Soyad,Gorev,CalismaTarihi,BarinakId
0,1,Büşra,Arslan,Veteriner,2022-01-03,1
1,2,Cem,Sarı,Bakici,2021-09-14,1
2,3,Nazan,Taş,Temizlik Gorevlisi,2021-03-09,1
3,4,Nesrin,Yavuz,Yonetici,2023-01-27,1
4,5,Serkan,Arslan,Gonullu,2021-03-12,1


In [10]:
dfVets.head()

,VetId,Ad,Soyad,Uzmanlik
0,1,Büşra,Arslan,Dahiliye
1,6,Fatih,Özkan,Anesteziyoloji
2,11,Elif,Çelik,Patoloji
3,16,Elvan,Savaş,Hayvan Besleme
4,21,Murat,Kurtuluş,Doğum ve Jinekoloji


In [11]:
dfInspections.head()

,MuayeneId,HayvanId,VeterinerId,Tarih,Teshis,Tedavi
0,1,2,1,2022-08-28,Diyabet,"İnsülin tedavisi, diyet ve egzersiz"
1,2,3,1,2024-03-23,Böbrek yetmezliği,"Diyet değişikliği, diyaliz, destekleyici bakım"
2,3,4,1,2022-02-03,Parazit enfeksiyonu,"Parazit önleyici ilaçlar, düzenli bakım"
3,4,5,1,2023-10-13,Kanser,"Kemoterapi, radyoterapi, cerrahi müdahale"
4,5,13,6,2023-04-18,Kanser,"Kemoterapi, radyoterapi, cerrahi müdahale"


### Hayvan Sheeti için listeler

In [12]:
animalIdList = pysqldf("SELECT HayvanId FROM dfAnimals")['HayvanId'].tolist()
animalNameList = pysqldf("SELECT Ad FROM dfAnimals")['Ad'].tolist()
animalSpecieList = pysqldf("SELECT Tur FROM dfAnimals")['Tur'].tolist()
animalAgeList = pysqldf("SELECT Yas FROM dfAnimals")['Yas'].tolist()
animalHealthConditionList = pysqldf("SELECT SaglikDurumu FROM dfAnimals")['SaglikDurumu'].tolist()
animalArrivalDateList = dfAnimals['GelisTarihi'].dt.strftime('%Y-%m-%d').tolist()
animalShelterIdList = pysqldf("SELECT BarinakId FROM dfAnimals")['BarinakId'].tolist()

### Barinak Sheeti için listeler

In [13]:
shelterIdList = pysqldf("SELECT BarinakId FROM dfShelters")['BarinakId'].tolist()
shelterNameList = pysqldf("SELECT Ad FROM dfShelters")['Ad'].tolist()
shelterLocList = pysqldf("SELECT Konum FROM dfShelters")['Konum'].tolist()
shelterCapacityList = pysqldf("SELECT Kapasite FROM dfShelters")['Kapasite'].tolist()
shelterPasswordList = pysqldf("SELECT Sifre FROM dfShelters")['Sifre'].tolist()

### Personel Sheeti için listeler

In [14]:
personelIdList = pysqldf("SELECT PersonelId FROM dfEmployees")['PersonelId'].tolist()
personelNameList = pysqldf("SELECT Ad FROM dfEmployees")['Ad'].tolist()
personelLastNameList = pysqldf("SELECT Soyad FROM dfEmployees")['Soyad'].tolist()
personelTaskList = pysqldf("SELECT Gorev FROM dfEmployees")['Gorev'].tolist()
personelArrivalDateList = dfEmployees['CalismaTarihi'].dt.strftime('%Y-%m-%d').tolist()
personelsToShelterList = pysqldf("SELECT BarinakId FROM dfEmployees")['BarinakId'].tolist()

### index.html'i aktif eden fonksiyon


In [15]:
@app.route('/')
def index():
  return render_template('index.html')

### user.html'i aktif eden fonksiyon

In [16]:
@app.route('/user')
def user():
    return render_template('user.html')

### shelter-owner-check.html'i aktif eden fonksiyon

In [17]:
@app.route('/shelter-owner-check')
def shelterOwnerCheck():
  return render_template('shelter-owner-check.html')

### shelter-owner.html'i aktif eden fonksiyon

In [18]:
@app.route('/shelter-owner', methods=['POST'])
def shelterOwner():
    global sifre
    barinakId = request.form.get('barinakId')
    sifre = request.form.get('sifre')

    if(barinakId == sifre):
        return render_template('shelter-owner.html')
    else:

        return render_template('shelter-owner-check.html')



### shelter-owner-animal-list'i aktif eden fonksiyon

In [19]:
@app.route('/shelter-owner-animal-list', methods=['GET', 'POST'])
def shelterOwnerAnimalList():
  q = f"SELECT * FROM dfAnimals WHERE BarinakId = {sifre}"
  dfResult = pysqldf(q)
  length = len(dfResult['BarinakId'].tolist())
  animals = []
  for a in range(length):
    row = dfResult.iloc[a]
    animals.append({
        'id': row['HayvanId'],
        'name': row['Ad'],
        'specie': row['Tur'],
        'age': row['Yas'],
        'health': row['SaglikDurumu'],
        'arrival': row['GelisTarihi'],
        'shelter': row['BarinakId']
    })

  return render_template('shelter-owner-animal-list.html', animals = animals)


### shelter-owner-employee-list'i aktif eden fonksiyon

In [20]:
@app.route('/shelter-owner-employee-list', methods=['GET', 'POST'])
def shelterOwnerEmployeeList():
  q = f"SELECT * FROM dfEmployees WHERE BarinakId = {sifre}"
  dfResult = pysqldf(q)
  length = len(dfResult['BarinakId'].tolist())
  employees = []
  for a in range(length):
    row = dfResult.iloc[a]
    employees.append({
            'id': row['PersonelId'],
            'firstName' : row['Ad'],
            'lastName' : row['Soyad'],
            'task' : row['Gorev'],
            'arrivalDate' : row['CalismaTarihi'],
            'shelterId' : row['BarinakId']
          })

  return render_template('shelter-owner-employee-list.html', employees = employees)


### shelter-owner-inspection-list'i aktif eden fonksiyon

In [21]:
@app.route('/shelter-owner-inspection-list', methods=['GET', 'POST'])
def shelterOwnerInspectionList():
  q = f"SELECT k.HayvanId, k.Ad, k.Tur, k.SaglikDurumu, m.MuayeneId, m.VeterinerId, m.Tarih, m.Teshis, m.Tedavi FROM dfAnimals k JOIN dfInspections m ON k.HayvanId = m.HayvanId WHERE k.BarinakId = {sifre} AND k.SaglikDurumu = 'Tedavi Ediliyor'"

  dfResult = pysqldf(q)
  length = len(dfResult['Ad'].tolist())
  inspections = []
  for a in range(length):
    row = dfResult.iloc[a]
    inspections.append({
        'id': row['MuayeneId'],
        'animalId': row['HayvanId'],
        'vetId': row['VeterinerId'],
        'date': row['Tarih'],
        'diagnosis': row['Teshis'],
        'treatment': row['Tedavi'],
    })

  return render_template('shelter-owner-inspection-list.html',inspections = inspections)


### add-animal'i aktif eden fonksiyon

In [22]:
@app.route('/add-animal', methods=['GET', 'POST'])
def addAnimal():
    diagnoses = [
    "Parvovirüs enfeksiyonu",
    "Distemper",
    "Kennel öksürüğü",
    "Lyme hastalığı",
    "Dermatit",
    "Kulak enfeksiyonu",
    "Parazit enfeksiyonu",
    "Gastrit",
    "Diyabet",
    "Kalp hastalığı",
    "Böbrek yetmezliği",
    "Hipotiroidi",
    "Diz kapağı çıkığı",
    "Kanser"
]
    treatments = [
    "Sıvı tedavisi, antiviral ilaçlar, destekleyici bakım",
    "Antiviral ve antibiyotik tedavisi, aşı",
    "Antibiyotikler, öksürük kesici ilaçlar, dinlenme",
    "Antibiyotikler, kene önleyici ilaçlar",
    "Topikal kortikosteroidler, antihistaminikler",
    "Antibiyotik damlalar, kulak temizliği",
    "Parazit önleyici ilaçlar, düzenli bakım",
    "Antasitler, diyet değişikliği, proton pompa inhibitörleri",
    "İnsülin tedavisi, diyet ve egzersiz",
    "İlaç tedavisi, diyet, egzersiz ve düzenli kontroller",
    "Diyet değişikliği, diyaliz, destekleyici bakım",
    "Tiroid hormonu replasman tedavisi",
    "Cerrahi müdahale, fizik tedavi, ağrı kesiciler",
    "Kemoterapi, radyoterapi, cerrahi müdahale"
]
    if request.method == 'POST':
        ad = request.form.get('ad')
        tur = request.form.get('tur')
        yas = int(request.form.get('yas'))
        saglik = request.form.get('saglik')
        gelis = request.form.get('gelis')
        cinsiyet = request.form.get('cinsiyet')
        
        if saglik not in ['Saglikli', 'Tedavi Ediliyor', 'Hasta']:
            return "Geçersiz sağlık durumu seçimi", 400
        
        excel = pd.ExcelFile(filePath + 'data.xlsx')
        dfHayvan = pd.read_excel(excel, sheet_name='hayvan')
        dfMuayene = pd.read_excel(excel, sheet_name='muayene')
        
        yeni_hayvan_id = dfHayvan['HayvanId'].max() + 1 if not dfHayvan.empty else 1
        
        yeni_hayvan = {
            'HayvanId': yeni_hayvan_id,
            'Ad': ad,
            'Tur': tur,
            'Yas': yas,
            'Cinsiyet': cinsiyet,
            'SaglikDurumu': saglik,
            'GelisTarihi': pd.to_datetime(gelis),
            'BarinakId': sifre
        }
        
        dfHayvan = pd.concat([dfHayvan, pd.DataFrame([yeni_hayvan])], ignore_index=True)
        if saglik == 'Tedavi Ediliyor':
            yeni_muayene_id = dfMuayene['MuayeneId'].max() + 1 if not dfMuayene.empty else 1
            yeni_muayene = {
                'MuayeneId': yeni_muayene_id,
                'HayvanId': yeni_hayvan_id,
                'VeterinerId': None,
                'Tarih': pd.to_datetime(gelis),
                'Teshis': random.choice(diagnoses),
                'Tedavi': random.choice(treatments)

            }
            dfMuayene = pd.concat([dfMuayene, pd.DataFrame([yeni_muayene])], ignore_index=True)
          
        with pd.ExcelWriter(filePath + 'data.xlsx', engine='openpyxl', mode='w') as writer:
            dfHayvan.to_excel(writer, sheet_name='hayvan', index=False)
            dfMuayene.to_excel(writer, sheet_name='muayene', index=False)
            dfShelters.to_excel(writer, sheet_name='barinak', index=False)
            dfEmployees.to_excel(writer, sheet_name = 'personel', index = False)
            dfVets.to_excel(writer, sheet_name = 'veteriner', index = False)
        exec(open('database.py').read())      
        return redirect(url_for('addAnimal'))

    return render_template('add-animal.html')


### add-personel'i aktif eden fonksiyon

In [23]:
@app.route('/add-personel', methods=['GET', 'POST'])
def add_personel():
    vetSpecialties = [
    "Dahiliye",            
    "Cerrahi",              
    "Doğum ve Jinekoloji",  
    "Parazitoloji",        
    "Mikrobiyoloji",       
    "Hayvan Besleme",       
    "Epidemiyoloji",       
    "Patoloji",             
    "Farmakoloji",         
    "Radyoloji",            
    "Rehabilitasyon",      
    "Anesteziyoloji",       
    "Zootekni",            
    "Yoğun Bakım",          
    "Acil Müdahale",        
    ]
    if request.method == 'POST':
        ad = request.form.get('ad')
        soyad = request.form.get('soyad')
        gorev = request.form.get('gorev')
        calisma_tarihi = request.form.get('calismaTarihi')

        newPersonelId = dfEmployees['PersonelId'].max() + 1
        excel = pd.ExcelFile(filePath + 'data.xlsx')
        dfPersonel = pd.read_excel(excel, sheet_name='personel')
        dfVeteriner = pd.read_excel(excel, sheet_name='veteriner')
        yeni_personel = {
            'PersonelId': newPersonelId,
            'Ad': ad,
            'Soyad': soyad,
            'Gorev': gorev,
            'CalismaTarihi': pd.to_datetime(calisma_tarihi),
            'BarinakId': sifre
        }
        dfPersonel = pd.concat([dfPersonel, pd.DataFrame([yeni_personel])], ignore_index=True)
        if gorev ==  'Veteriner':
            yeni_veteriner = {
            'VetId': newPersonelId,
            'Ad':ad,
            'Soyad':soyad,
            'Uzmanlik':random.choice(vetSpecialties)
            }
            dfVeteriner = pd.concat([dfVeteriner, pd.DataFrame([yeni_veteriner])], ignore_index=True)


        with pd.ExcelWriter(filePath + 'data.xlsx', engine='openpyxl', mode='w') as writer:
            dfAnimals.to_excel(writer, sheet_name='hayvan', index=False)
            dfInspections.to_excel(writer, sheet_name='muayene', index=False)
            dfShelters.to_excel(writer, sheet_name='barinak', index=False)
            dfPersonel.to_excel(writer, sheet_name='personel', index=False)
            dfVeteriner.to_excel(writer, sheet_name='veteriner', index=False)
        exec(open('database.py').read())

    return render_template('add-personel.html')


### id-to-animal-list.html'i aktif eden fonksiyon

In [24]:
@app.route('/id-to-animal-list', methods=['GET', 'POST'])
def idToAnimalList():
  return render_template('id-to-animal-list.html')

### id-to-animal-list-result.html'i aktif eden fonksiyon

In [25]:
@app.route('/id-to-animal-list-result',methods = ['GET','POST'])
def idToAnimalListResult():
  i = request.form.get('id')
  i = int(i)
  q = f"SELECT * FROM dfAnimals WHERE HayvanId = {i}"
  dfResult = pysqldf(q)
  row = dfResult.iloc[0]
  animals = []
  animals.append({
        'id': row['HayvanId'],
        'name': row['Ad'],
        'specie': row['Tur'],
        'age': row['Yas'],
        'health': row['SaglikDurumu'],
        'arrival': row['GelisTarihi'],
        'shelter': row['BarinakId']
        })
  return render_template('id-to-animal-list-result.html', animals = animals)

  

### id-to-shelter-list.html'i aktif eden fonksiyon

In [26]:
@app.route('/id-to-shelter-list', methods=['GET', 'POST'])
def idToShelterList():
  return render_template('id-to-shelter-list.html')

### id-to-shelter-list-result.html'i aktif eden fonksiyon

In [27]:
@app.route('/id-to-shelter-list-result',methods = ['GET','POST'])
def idToShelterListResult():
  i = request.form.get('id')
  i = int(i)
  q = f"SELECT * FROM dfShelters WHERE BarinakId = {i}"
  dfResult = pysqldf(q)
  row = dfResult.iloc[0]
  shelters = []
  
  shelters.append({
          'id': row['BarinakId'],
          'name' : row['Ad'],
          'location' : row['Konum'],
          'capacity' : row['Kapasite']
        })
  return render_template('id-to-shelter-list-result.html', shelters = shelters)

### animal-list.html'i aktif eden fonksiyon

In [28]:
@app.route('/animal-list')
def animalList():
    q = "SELECT * FROM dfAnimals"
    dfResult = pysqldf(q)
    animals = []
    for i in range(len(animalIdList)):
        row = dfResult.iloc[i]
        animals.append({
            'id': row['HayvanId'],
            'name': row['Ad'],
            'specie': row['Tur'],
            'age': row['Yas'],
            'health': row['SaglikDurumu'],
            'arrival': row['GelisTarihi'],
            'shelter': row['BarinakId']
        })
    return render_template('animal-list.html', animals=animals)


### shelter-list.html'i aktif eden fonksiyon

In [29]:
@app.route('/shelter')
def shelterList():
  q = "SELECT * FROM dfShelters"
  dfResult = pysqldf(q)
  shelters = []
  for i in range(len(shelterIdList)):
    row = dfResult.iloc[i]
    shelters.append({
      'id': row['BarinakId'],
      'name' : row['Ad'],
      'location' : row['Konum'],
      'capacity' : row['Kapasite']
    })
  return render_template('shelter-list.html', shelters = shelters) 

### shelter-id-to-employee-list.html'i aktif eden fonksiyon

In [30]:
@app.route('/shelter-id-to-employee-list', methods=['GET', 'POST'])
def shelterIdToEmployeeList():
  return render_template('shelter-id-to-employee-list.html')

### shelter-id-to-employee-list-result.html'i aktif eden fonksiyon

In [31]:
@app.route('/shelter-id-to-employee-list-result',methods = ['GET','POST'])
def shelterIdToEmployeeListResult():
  i = request.form.get('id')
  i = int(i)
  q = f"SELECT * FROM dfEmployees WHERE BarinakId = {i}"
  dfResult = pysqldf(q)
  employees = []
  length = len(dfResult['PersonelId'].tolist())
  for i in range(length):
    row = dfResult.iloc[i]
    employees.append({
            'id': row['PersonelId'],
            'firstName' : row['Ad'],
            'lastName' : row['Soyad'],
            'task' : row['Gorev'],
            'arrivalDate' : row['CalismaTarihi'],
            'shelterId' : row['BarinakId']
          })
  return render_template('shelter-id-to-employee-list-result.html', employees = employees)


### Uygulamayı çalıştır

In [ ]:
if __name__ == '__main__':
    webbrowser.open('http://127.0.0.1:5000')
    app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [28/May/2025 13:31:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2025 13:31:37] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [28/May/2025 13:31:37] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [28/May/2025 13:31:44] "GET /user HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2025 13:31:44] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [28/May/2025 13:31:46] "GET /animal-list HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2025 13:31:46] "GET /static/lists.css HTTP/1.1" 304 -
127.0.0.1 - - [28/May/2025 13:32:07] "GET /shelter HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2025 13:32:07] "GET /static/lists.css HTTP/1.1" 304 -
127.0.0.1 - - [28/May/2025 13:32:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2025 13:32:15] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [28/May/2025 13:32:17] "GET /user HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2025 13:32:17] "GET /static/index.css HTTP/1.1" 304 -
127.0.0.1 - - [28/May/2025 13:

    VetId        Ad     Soyad             Uzmanlik
0       1     Ferit   Bozkurt  Doğum ve Jinekoloji
1       6     Şirin  Kurtuluş        Epidemiyoloji
2      11     Burcu     Özkan       Anesteziyoloji
3      16      Emre     Özkan        Mikrobiyoloji
4      21      Ebru      Kurt              Cerrahi
5      26     Merve    Öztürk       Hayvan Besleme
6      31      Suat      Kara             Dahiliye
7      36     Selin     Çetin          Farmakoloji
8      41     Selin      Kurt            Radyoloji
9      46     Necla     Doğan       Rehabilitasyon
10     51      Ömer    Kalkan       Hayvan Besleme
11     56     Burak     Çelik          Yoğun Bakım
12     61  Süleyman       Gök             Dahiliye
13     66    Gökhan   Özdemir             Dahiliye
14     71    Feride      Sarı        Acil Müdahale
15     76    Furkan     Özkan        Epidemiyoloji
16     81      Eren    Yalçın          Farmakoloji
17     86      Suat      Sarı       Hayvan Besleme
18     91     Özkan       Koç  

127.0.0.1 - - [28/May/2025 13:34:58] "GET /add-personel HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2025 13:34:58] "GET /static/form.css HTTP/1.1" 304 -
127.0.0.1 - - [28/May/2025 13:35:21] "POST /add-personel HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2025 13:35:21] "GET /static/form.css HTTP/1.1" 304 -


    VetId        Ad     Soyad             Uzmanlik
0       1     Ferit   Bozkurt  Doğum ve Jinekoloji
1       6     Şirin  Kurtuluş        Epidemiyoloji
2      11     Burcu     Özkan       Anesteziyoloji
3      16      Emre     Özkan        Mikrobiyoloji
4      21      Ebru      Kurt              Cerrahi
5      26     Merve    Öztürk       Hayvan Besleme
6      31      Suat      Kara             Dahiliye
7      36     Selin     Çetin          Farmakoloji
8      41     Selin      Kurt            Radyoloji
9      46     Necla     Doğan       Rehabilitasyon
10     51      Ömer    Kalkan       Hayvan Besleme
11     56     Burak     Çelik          Yoğun Bakım
12     61  Süleyman       Gök             Dahiliye
13     66    Gökhan   Özdemir             Dahiliye
14     71    Feride      Sarı        Acil Müdahale
15     76    Furkan     Özkan        Epidemiyoloji
16     81      Eren    Yalçın          Farmakoloji
17     86      Suat      Sarı       Hayvan Besleme
18     91     Özkan       Koç  

127.0.0.1 - - [28/May/2025 13:35:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/May/2025 13:35:22] "GET /static/index.css HTTP/1.1" 304 -


In [ ]:
exec(open('database.py').read())